# Оглавление

* [1. Добавление признаков](#chapter1)
  
* [2. Построение гипотез](#chapter2)
  
* [3. Построение модели](#chapter3)
  
* [4. Выводы](#chapter4)

In [1]:
# Устанавливаем необходимые библиотеки
!pip install -q pandas scipy pycountry_conver

ERROR: Could not find a version that satisfies the requirement pycountry_conver (from versions: none)
ERROR: No matching distribution found for pycountry_conver

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np

import scipy.stats as stats

from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

import pycountry_convert as pc

In [3]:
import warnings

warnings.filterwarnings('ignore')

In [4]:
file_path = 'dataset/ds_salaries_cleared.csv'

In [5]:
df = pd.read_csv(file_path)

# 1. Добавление признаков <a class="anchor" id="chapter1"></a>

In [6]:
def get_continent(country_code):
    try:
        continent_code = pc.country_alpha2_to_continent_code(country_code)
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        return continent_name
    except KeyError:
        return 'Unknown'

In [7]:
# Создание нового признака 'salary_bracket'
df['salary_bracket'] = pd.cut(df['salary_in_usd'], bins=[0, 70000, 180000, float('inf')], labels=['low', 'medium', 'high'])

# Преобразование типа занятости в категории
df['job_type'] = df['employment_type'].map(lambda x: 'full_time' if x == 'FT' else 'part_time' if x == 'PT' else 'contract' if x == 'C' else 'freelance')

# Преобразование стран в регионы
df['region'] = df['company_location'].apply(get_continent)

# Создание бинарного признака удалённой работы
df['is_remote'] = df['remote_ratio'].apply(lambda x: 1 if x > 0 else 0)

# Добавление больше признаков
df['experience_years'] = 2024 - df['work_year']

In [8]:
df['region'].value_counts()

North America    2024
Europe            402
Asia               94
South America      24
Oceania            18
Africa             14
Name: region, dtype: int64

# 2. Гипотезы <a class="anchor" id="chapter2"></a>

Выдвинуты следуюие гипотезы:

* Гипотеза 1: Средняя зарплата сотрудников, работающих удаленно, отличается от средней зарплаты сотрудников, работающих в офисе

* Гипотеза 2: Средняя зарплата сотрудников в Северной Америке отличается от средней зарплаты сотрудников в других регионах

* Гипотеза 3: Средняя зарплата сотрудников с полной занятостью (FT) отличается от средней зарплаты сотрудников с другими типами занятости (PT, C, FL)

In [9]:
# Гипотеза 1: Влияние удаленной работы на уровень зарплаты
remote_salary = df[df['is_remote'] == 1]['salary_in_usd']
office_salary = df[df['is_remote'] == 0]['salary_in_usd']
t_stat, p_val = stats.ttest_ind(remote_salary, office_salary)
print(f"t-статистика: {t_stat}, p-значение: {p_val}")

t-статистика: -7.474501770973419, p-значение: 1.056070355362127e-13


Поскольку p-значение значительно меньше 0.05, мы отвергаем нулевую гипотезу. 

Это означает, что существует статистически значимое различие в уровнях зарплаты между сотрудниками, работающими удаленно, и сотрудниками, работающими в офисе. 

Отрицательная t-статистика указывает на то, что средняя зарплата у сотрудников, работающих в офисе, выше, чем у сотрудников, работающих удаленно.

In [10]:
# Гипотеза 2: Влияние региона на уровень зарплаты
north_america_salary = df[df['region'] == 'North America']['salary_in_usd']
other_regions_salary = df[df['region'] != 'North America']['salary_in_usd']
t_stat, p_val = stats.ttest_ind(north_america_salary, other_regions_salary)
print(f"t-статистика: {t_stat}, p-значение: {p_val}")

t-статистика: 31.028509133291664, p-значение: 7.501739986581016e-180


Поскольку p-значение значительно меньше 0.05, мы отвергаем нулевую гипотезу. 

Это означает, что существует статистически значимое различие в уровнях зарплаты между сотрудниками в Северной Америке и сотрудниками в других регионах. 

Положительная t-статистика указывает на то, что средняя зарплата в Северной Америке выше, чем в других регионах.

In [11]:
# Гипотеза 3: Влияние типа занятости на уровень зарплаты
ft_salary = df[df['employment_type'] == 'FT']['salary_in_usd']
ot_salary = df[df['employment_type'] != 'FT']['salary_in_usd']
f_stat, p_val = stats.f_oneway(ft_salary, ot_salary)
print(f"F-статистика: {f_stat}, p-значение: {p_val}")

F-статистика: 43.22171352104709, p-значение: 5.896539760425233e-11


Поскольку p-значение значительно меньше 0.05, мы отвергаем нулевую гипотезу. 

Это означает, что существует статистически значимое различие в уровнях зарплаты между сотрудниками с полной занятостью (FT) и другими типами занятости (PT, C, FL). 

F-статистика указывает на значительное различие в зарплатах между этими группами.

Все три гипотезы подтвердились, что указывает на наличие значимых различий в зарплатах, обусловленных режимом работы (удаленно или в офисе), регионом и типом занятости.

# 3. Машинное обучение <a class="anchor" id="chapter3"></a>

Предскажем зарплату сотрудников - поле **salary_in_usd**

In [12]:
# Копирование данных и удаление ненужных столбцов
salary_data_copy = df.copy()
salary_data_copy = salary_data_copy.drop(['job_title', 'employee_residence', 'salary_currency', 'region'], axis=1)

# Преобразование категориальных признаков в числовые
salary_data_copy = pd.get_dummies(salary_data_copy, dtype='int')

In [13]:
# Разделение данных на обучающую и тестовую выборку
X = salary_data_copy.drop('salary_in_usd', axis=1)
y = salary_data_copy['salary_in_usd']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

In [14]:
# Применение кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=10)
models = []

# Обучение на всех фолдах
for train_index, test_index in kf.split(x_train):
    X_train_fold, X_test_fold = x_train.iloc[train_index], x_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    model = LinearRegression()
    model.fit(X_train_fold, y_train_fold)
    models.append(model)

# Получение предсказаний от каждой модели и их объединение
predictions = [model.predict(x_test) for model in models]
combined_predictions = np.vstack(predictions).T

# Разделение объединенных предсказаний и целевой переменной на обучающие и тестовые наборы для мета-модели
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(combined_predictions, y_test, test_size=0.2, random_state=42)

In [15]:
# Обучение мета-модели (Linear Regression)
meta_model = LinearRegression()
meta_model.fit(X_train_5, y_train_5)

# Оценка мета-модели
y_pred_meta = meta_model.predict(X_test_5)
meta_model_score = mean_squared_error(y_test_5, y_pred_meta, squared=False)
print('RMSE мета-модели: ', meta_model_score)

RMSE мета-модели:  26442.490421748593


In [16]:
# Гиперпараметры для подбора
param_grids = {
    'Ridge Regression': {
        'alpha': [0.01, 0.1, 1, 10, 100]
    },
    'Lasso Regression': {
        'alpha': [0.01, 0.1, 1, 10, 100]
    },
    'Decision Tree': {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10]
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5]
    }
}

In [17]:
# Обучение дополнительных моделей с подбором гиперпараметров
additional_models = {
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42)
}

In [18]:
%%time
results = []

for model_name, model in additional_models.items():
    grid_search = GridSearchCV(model, param_grids[model_name], cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(x_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(x_test)
    rmse = mean_squared_error(y_test, y_pred, squared=False)
    
    results.append({
        'Model': model_name,
        'Best Parameters': grid_search.best_params_,
        'RMSE': rmse
    })

CPU times: total: 797 ms
Wall time: 22.6 s


In [19]:
# Создание DataFrame с результатами
results_df = pd.DataFrame(results)

# Вывод таблицы результатов
results_df

,Model,Best Parameters,RMSE
0,Ridge Regression,{'alpha': 10},29890.076433
1,Lasso Regression,{'alpha': 100},29780.334832
2,Decision Tree,"{'max_depth': 20, 'min_samples_split': 5}",4828.493755
3,Random Forest,"{'max_depth': 20, 'min_samples_split': 2, 'n_e...",3903.988744
4,Gradient Boosting,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",3499.911469


# 4. Выводы <a class="anchor" id="chapter4"></a>

Результаты моделей

Линейные модели (Ridge и Lasso) показывают RMSE около 30,000, что является средним результатом

Деревья решений и ансамблевые модели (Decision Tree, Random Forest, Gradient Boosting) значительно снижают RMSE до 3,500-4,500, указывая на их превосходство в предсказании зарплат

Выводы по гипотезам

Удаленная работа: Зарплаты сотрудников, работающих в офисе, значительно выше, чем у сотрудников, работающих удаленно

Регион: Сотрудники в Северной Америке получают значительно больше, чем сотрудники в других регионах

Тип занятости: Полная занятость (FT) приводит к значительно более высоким зарплатам по сравнению с другими типами занятости

Общий вывод

Модели на основе деревьев и ансамблей являются наиболее эффективными для предсказания зарплат

Анализ гипотез подтвердил, что удаленная работа, регион и тип занятости существенно влияют на уровень зарплаты